<table style="width: 100%; border-collapse: collapse;" border="0">
<tr>
<td><b>Created:</b> Tuesday 31 January 2017</td>
<td style="text-align: right;"><a href="https://www.github.com/rhyswhitley/fire_limitation">github.com/rhyswhitley/fire_limitation</td>
</tr>
</table>

<div>
<center>
<font face="Times">
<br>
<h1>Quantifying the uncertainity of a global fire limitation model using Bayesian inference</h1>
<h2>Part 2: Bayesian inference</h2>
<br>
<br>
<sup>1,* </sup>Douglas Kelley, 
<sup>2 </sup>Ioannis Bistinas,
<sup>3 </sup>Rhys Whitley,
<sup>4 </sup>Chantelle Burton, 
<sup>1 </sup>Tobias Marthews, 
<sup>6, 7 </sup>Ning Dong
<br>
<br>
<br>
<sup>1 </sup>Centre for Ecology and Hydrology, Maclean Building, Crowmarsh Gifford, Wallingford, Oxfordshire, United Kingdom
<br>
<sup>2 </sup>Vrije Universiteit Amsterdam, Faculty of Earth and Life Sciences, Amsterdam, Netherlands
<br>
<sup>3 </sup>Natural Perils Pricing, Commercial & Consumer Portfolio & Pricing, Suncorp Group, Sydney, Australia
<br>
<sup>4 </sup>Met Office United Kingdom, Exeter, United Kingdom
<br>
<sup>5 </sup>Centre for Past Climate Change and School of Archaeology, Geography and Environmental Sciences (SAGES), University of Reading, Reading, United Kingdom 
<br>
<sup>6 </sup>Department of Biological Sciences, Macquarie University, North Ryde, NSW 2109, Australia 
<br>
<br>
<h3>Summary</h3>
<hr>
<p> 
This notebook aims to quantify the model parameters of a global fire model (defined below). The model is driven by a number of covariates (X<sub>i=1, 2, ... M</sub>) that describe: cropland, pasture and urban area footprints; frequency of lightening ignitions, population density, net primary productivity (NPP) and <i>Alpha</i>, a proxy measure of available soil moisture in the root zone. The model attempts to predict the impact of fire through burnt area and is thus the model target (Y).
</p>
<br>
<br>
<br>
<i>Python code and calculations below</i>
<br>
</font>
</center>
<hr>
</div>

### Model description
The model considers percentage of burnt area to be the joint product of a set of conditions that modulate fire through fuel load, ignitions, moisture and supression. Each control assumes some equilibrium point that desribes the optimal conditions for fire, that may be proportionally modified through some empirical relationship. These are briefly outlined below for the sake of comprehension in this notebook, but can be referred to in more detail in the model protocol located in the docs/ folder (<a href='file:///localhost/../docs/Model_description.pdf'>model protocol</a>).

\begin{eqnarray}
    F_{burn} &=& \prod_{i}S(x_{i}) \\[1em]
\end{eqnarray}

Where $S(x_{i})$ representes some measure of fire conditions by $i =$ fuel, moisture, ignitions and anthropagenic supression, and is describe by a sigmoid:

\begin{equation}
    S(x_{i=fuel, moist, ignite, suppr}) = \frac{1}{1 + \exp\{-b\cdot(x_i-a)\}}
\end{equation}

The $fuel$ sigmoid considers only fractional vegetation cover and therefore have no hyper-parameters. Sigmoids $moist$, $ignite$ and $suppr$ describe an aggregation of other climate and land-use covariates. Because these sigmoids are influenced by an aggregation of different drivers, they are influenced in turn by different sets of hyper-parameters; these are now described below.  

#### Fuel load covariate (no hyper-parameters)
\begin{equation}
    x_{fuel} = FPC^{p} \cdot (v_{fuel} \cdot (\alpha_{max}/\alpha -1) + 1)/(1+v_{fuel}) 
\end{equation}

#### Moisture covariate
\begin{equation}
    x_{moist} = \alpha + v_M \cdot EMC    
\end{equation}

where $EMC$ is the equilibrium moisture content.
#### Ignition covariate 
\begin{equation}
    x_{ignite} = Lightn + v_p\cdot A_{pasture} + v_{d1}\cdot\rho_{population}
\end{equation}

Where $Lightn$ is the number of cloud-to-ground lightning strikes, modified  as per Kelley et al. 2014.

#### Supression covariate 
\begin{equation}
    x_{supress} = A_{urban} + v_C\cdot A_{Crop} + v_{d2}\cdot\rho_{population} 
\end{equation}

This leaves 17 free parameters that need to be optimised against observations of burnt area.  

### Load libraries

In [4]:
import os
from   io     import StringIO
import numpy  as np
import pandas as pd
import csv

import pymc3  as pm3 
from   pymc3.backends import SQLite
from   scipy  import optimize
from   theano import tensor as tt

import matplotlib.pyplot as plt

# setup nice plotting
plt.style.use('ggplot')
%matplotlib inline

# paths and parameters
                    # Full model
sample_pc     = 2 # = 10 # Percentage of grid cells samples (largest = 100%) - training set
nIterations   = 5000 # = 10000 # no. times it samples the posterior
nJobs         = 2   # = 2 # no. times it runs through optimisation proceedure = sample_pc * nJobs
#chains per job
nChains       = 2   # = 3
#how many times will you smple the postirior
n_posterior_sample = 100 # = 100 for analysis = 1000 for publication.


datPath       = "../data/globfire.csv"
param_outpath = '../outputs/params-test.csv'
trace_db      = '../data/firemodel_trace.db'

In [5]:
def npLog0 (x):
    return [np.log(i) if i > 0 else 0.0000001 for i in x]

## 2.1 Fire limitation model definition

Could possibly contain this in a class object, but I'm not sure theano can instantiate the object to be used by the GPU. If I've made absolutely no sense just then, then I would leave the following as is.

In [7]:
from pdb import set_trace as browser
def fuel_load(fuel, alphaMax, fp, fpg):
    """
    Definition to describe fuel load: while return the input; capability to be modified later.
    """
    #falpha = (1/fp) + (alphaMax/fpg)
    #return fuel**fp
    return (fuel**fp) * (fpg * alphaMax + 1) / (1 + fpg)
    #return (fuel**fp + fpg * alphaMax) / (1 + fpg)

def moisture(alpha, emc, tree, cM, cMT):
    """
    Definition to describe moisture
    """
    return (alpha + 0.01*cM*emc + 0.01*cMT * tree) / (1 + cM + cMT)


def ignition(lightning, pasture_area, pop_density, cP, cD1):
    """
    Definition for the measure of ignition
    """
    ignite = lightning + cP*pasture_area + cD1*pop_density
    #try:
    #    ignite = npLog0(ignite)
    #except:
    #    ignite = np.log(ignite)
    return ignite

def supression(crop_area, pop_density, cD2):
    """
    Definition for the measure of fire supression
    """
    return crop_area + cD2*pop_density

def tt_sigmoid(x, k, x0):
    """
    Sigmoid function to describe limitation using tensor
    """
    return 1.0/(1.0 + tt.exp(-k*(x - x0)))

## 2.2 Import data

Load data and do any necessary transformation needed for the Bayesian modelling framework. Rows are defined as a fraction of total data points (above). For full optimiatizaion, we set at 10%, but for testing purposes I've limited the number of rows I'm importing to 2%.

In [8]:
def load_with_buffer(filename, line_select, **kwargs):
    s_buf = StringIO()
    line_select = np.sort(line_select)
    with open(filename) as file:
        count = -1
        lineN = -1
        for line in file:
            lineN += 1
            if lineN == 0 or lineN == line_select[count]:
                s_buf.write(line)
                count += 1
                if count == len(line_select): break
            
    s_buf.seek(0)
    df = pd.read_csv(s_buf,**kwargs)
    return df

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f): pass
    return i + 1

DATAPATH = os.path.expanduser(datPath)

nlines      = file_len(DATAPATH)
npoints     = round(sample_pc * nlines / 100)
line_select = np.random.choice(range(0, nlines), npoints, False)
fd          = load_with_buffer(DATAPATH, line_select)

In [10]:
fd['fuel'] = npLog0(fd['NPP'].values)
fd['vegCover'] = (100.0 - fd['bareground'])/100.0
fd['alphaMax'] = fd['alpha_'] - 1.0

Do a sanity check to make sure our data has imported correctly.

In [11]:
fd.info()
fd.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204666 entries, 0 to 204665
Data columns (total 19 columns):
alpha                  204666 non-null float64
alpha_                 204666 non-null float64
bareground             204666 non-null float64
cropland               204666 non-null float64
emc                    204666 non-null float64
fire                   204666 non-null float64
Hr                     204666 non-null float64
lightning_ignitions    204666 non-null float64
NPP                    204666 non-null float64
pasture                204666 non-null float64
population_density     204666 non-null float64
Prc                    204666 non-null float64
Tas                    204666 non-null float64
treecover              204666 non-null float64
urban_area             204666 non-null float64
Wet                    204666 non-null float64
fuel                   204666 non-null float64
vegCover               204666 non-null float64
alphaMax               204666 non-null floa

,alpha,alpha_,bareground,cropland,emc,fire,Hr,lightning_ignitions,NPP,pasture,population_density,Prc,Tas,treecover,urban_area,Wet,fuel,vegCover,alphaMax
0,0.557876,1.414595,91.0,0.0,60.283752,0.0,53.055405,0.0,105.361115,0.000000,0.000000,34.400002,7.5,2.0,0.0,0.397097,4.657394e+00,0.09,0.414595
1,1.243895,1.176877,100.0,0.0,55.069386,0.0,54.714714,0.0,0.000000,0.000000,0.000000,27.700001,-0.6,0.0,0.0,0.311935,1.000000e-07,0.00,0.176877
2,0.245052,1.447964,100.0,0.0,34.145264,0.0,66.015167,0.0,104.121696,0.000000,0.000000,7.800000,4.6,0.0,0.0,0.156452,4.645560e+00,0.00,0.447964
3,0.421449,1.427807,100.0,0.0,49.884430,0.0,54.688221,0.0,6.486486,0.000000,0.000000,21.799999,2.5,0.0,0.0,0.251935,1.869721e+00,0.00,0.427807
4,1.244967,1.003530,95.0,0.0,55.443096,0.0,66.946335,0.0,147.793472,0.000000,0.000000,34.299999,4.4,1.0,0.0,0.323871,4.995816e+00,0.05,0.003530
5,0.391838,1.712254,100.0,0.0,41.131245,0.0,65.175766,0.0,79.762375,0.000000,0.000000,14.400000,2.0,0.0,0.0,0.196129,4.379052e+00,0.00,0.712254
6,0.244040,1.318328,100.0,0.0,43.236794,0.0,77.163040,0.0,140.166672,0.000000,0.000000,10.000000,4.5,0.0,0.0,0.227097,4.942832e+00,0.00,0.318328
7,1.245063,1.013221,91.0,0.0,65.203506,0.0,70.040298,0.0,527.951477,0.000000,0.000000,69.099998,4.5,1.0,0.0,0.472581,6.269004e+00,0.09,0.013221
8,0.245869,1.605815,69.0,0.0,46.567947,0.0,52.929966,0.0,244.106064,0.000000,0.000000,8.600000,10.9,3.0,0.0,0.255806,5.497603e+00,0.31,0.605815
9,0.459866,1.780636,100.0,0.0,49.105770,0.0,75.445908,0.0,108.217392,0.000000,0.000000,20.500000,4.6,0.0,0.0,0.270645,4.684142e+00,0.00,0.780636


## 2.3 Baysian framework

A simple explanation of Baye's law is:

\begin{equation}
    P(\beta|X) \propto P(\beta)\cdot P(X|\beta)
\end{equation}

where $X$ is our data (observations of some arbitrary system), and $\beta$ our set of unexplained parameters that describe the reponse of our _proposed understanding_ of this system as it varies with $X$.

### 2.3.1 Prior definitions
Because I have no idea what the uncertainty on the hyper parameters should look like (beyond $\beta> 0$), I've set them all as uniform, where bounds are either physical limits of the variable in question, or set generously beyound what is realistic plausable. Some of them can possibly be describe as exponential or half-normal, due to the physical nature of $\beta$, but we can play around with that later.

\begin{eqnarray}
    P(\beta) &=& \prod_{i=1}^{4}P(a_i)\prod_{i=1}^{4}P(b_i)\cdot P(\sigma)\cdot P(v_c)P(v_p)P(v_{d,1})P(v_{d,2}) \\[1.5em]
    P(a) = P(b) = P(\sigma) &=& \mathcal{N}(0, 1) \\[1em]
    P(v_c) = P(v_p) = P(v_{d,1}) = P(v_{d,2}) &=& \mathcal{U}(\beta_{\min}, \beta_{\max}) \\[1.5em]
\end{eqnarray}

I'm not totally sure about the maths above being right, but it's just to show that _full_ prior is normal. Important, because we'll also describe the error (likelihood) as normal, such that the posterior is therefore normal (conjugate); i.e. $\mathcal{N}\times\mathcal{N}=\mathcal{N}$ (expansion happens in the mean of the exponent). 

Back to the code.., `pymc3` is quite funky in that it allows me to create an empty `Model()` object and just add things to it as I need them using a `with` statement. I've called our Bayesian model `fire_error` as that is what we are trying to Quantify.



In [12]:
with pm3.Model() as fire_error:
    
# first for the sigmoids  
    fuel_x0        = pm3.Uniform('fuel_x0'       ,    -1.0,  2.0)
    fuel_k         = pm3.Uniform('fuel_k'        ,     0.0, 1000.0)
    fuel_pw        = pm3.Uniform('fuel_pw'       ,     1.0, 20.0)
    fuel_pg        = pm3.Uniform('fuel_pg'       ,     0.0, 1.0)
    
    moisture_x0    = pm3.Uniform('moisture_x0'   ,    -1.0,  2.0)
    moisture_k     = pm3.Uniform('moisture_k'    ,     0.0, 1000.0)
    
    ignition_x0    = pm3.Uniform('igntions_x0'   ,    -4.0, 4.0)
    ignition_k     = pm3.Uniform('igntions_k'    ,     0.0, 1000.0)
    
    suppression_x0 = pm3.Uniform('suppression_x0',  -100.0,  200.0)
    suppression_k  = pm3.Uniform('suppression_k' ,     0.0, 10.0)
    
    max_f          = pm3.Uniform('max_f'         ,     0.0,    1.0)
    #pow_f          = pm3.Uniform('pow_f'         ,     0.0,    1.0)
# now for the hyper-parameters that describe the independent fire condition covariates
    cM  = pm3.Uniform('cM' , 0.0, 1e4)
    cMT = pm3.Uniform('cMT', 0.0, 1e4)
    #cL  = pm3.Uniform('cL' , 0, 1e4)
    cP  = pm3.Uniform('cP' , 0.0, 1e4)
    cD1 = pm3.Uniform('cD1', 0.0, 1e4)
    cD2 = pm3.Uniform('cD2', 0.0, 1e4)
    #cDmax = pm3.Uniform('cDmax', 0, 1e4)
# describe the standard deviation in the error term
    sigma = pm3.HalfNormal('sigma', sd=1)

### 2.3.2 Likelihood definition

For the sake of simplicity (and because I don't really know any better), we define the model error as normally distributed (i.i.d.) although it most likely isn't. We could make this more complicated later by defining the error as heteroscedastic, but I wouldn't bother with that until we have some idea of the convergence. We're describing the error (observations minus model predictions) as follows:

\begin{eqnarray}
    P(X|\beta) &=& \mathcal{N}(F_{burn}, \sigma) \\[1em]
    \mathcal{N}(F_{burn}, \sigma) &=& \frac{N}{\sigma\sqrt{2\pi}}\exp\left\{\sum_{i=1}^{N}\left(\frac{y_i - F_{burn, i}}{\sigma_i}\right)^2\right\}
\end{eqnarray}

where $y_i$ is a set of observations we're attempting to optimise on. Below is the code that describes the above:

In [ ]:
with fire_error:
    
    # transform hyper-covariates 
    f_fuel        = fuel_load(fd["vegCover"].values, fd["vegCover"].values, fuel_pw, fuel_pg)
    
    f_moisture    = moisture(fd["alpha"].values, fd["emc"].values, fd["treecover"].values, cM, cMT)
    
    f_ignition    = ignition(fd["lightning_ignitions"].values, \
                             fd["pasture"].values, \
                             fd["population_density"].values, \
                             cP, cD1)
    
    f_suppression = supression(fd["cropland"].values, \
                               fd["population_density"].values, \
                               cD2)
    
    # burnt area is assumed to be the product of the 4 sigmoids
    
    prediction = max_f * np.product([tt_sigmoid(f_fuel, fuel_k, fuel_x0),
                             tt_sigmoid(f_moisture, - moisture_k, moisture_x0),
                             tt_sigmoid(f_ignition, ignition_k, ignition_x0),
                             tt_sigmoid(f_suppression, - suppression_k, suppression_x0)])
                  
    # calculate the error between observed and predicted burnt area
    error = pm3.Normal('error', mu=prediction, sd=sigma, observed=fd['fire'].values)

### 2.3.3 Posterior sampling

Because it is nigh impossible to determine the posterior solution analytically we will instead sample the information space to **infer** the posterior solutions for each of the model parameters. In this case we are using a Metropolis-Hasting step MCMC.

I've tried using No-U-Turn (NUTS) sampling (which is the new kid on the block), but there are issues with it's current implementation in pymc3 (see github repo issues). Can use it once problems are ironed out - but TBH it doesn't matter if we're getting a reasonable convergence.

In [ ]:
nChains = nChains* nJobs
with fire_error:
    
    # help the sampling out by quickly finding an optimal start position
    start = pm3.find_MAP(model=fire_error.model, fmin=optimize.fmin_powell)
    
    # set the step-method (criteria algorithm for moving around information space)
    step = pm3.Metropolis()
    
    # save our sampling to disk so we can access it later
    #db_save = SQLite(trace_db)
    
    # do the sampling
    mcmc_traces = pm3.sample(nIterations * nChains * nJobs, step=step, start=start, njobs= nJobs, chains = nChains) 

/users/global/doukel/anaconda3/lib/python3.6/site-packages/pymc3/tuning/starting.py:92: UserWarning: In future versions, set the optimization algorithm with a string. For example, use `method="L-BFGS-B"` instead of `fmin=sp.optimize.fmin_l_bfgs_b"`.
  warnings.warn('In future versions, set the optimization algorithm with a string. '
logp = 4.3087e+05:   1%|          | 32/5000 [00:01<04:32, 18.24it/s]

In [ ]:
pm3.traceplot(mcmc_traces);

## Output paramaters
The iterations at the start are just letting the optimization settle. So we will only sample to last 50% of iterations for futher analysis. We also expor these to csv, which others can use to do their own analysis.

In [ ]:
varnames = mcmc_traces.varnames


def cutLastX(varname, mcmc_traces, ncut = 50):
    vals = mcmc_traces.get_values(varname)
    def subcut(vals, r, ncut = 50):
        cut_np = (r+1) * round(len(vals)/nChains)
        ncut = round(len(vals) * ncut / (nChains *100))
        return vals[(cut_np - ncut):cut_np]
    vals = [subcut(vals, r) for r in range(nChains)]
    return np.array(vals).flatten()

vals = [cutLastX(i, mcmc_traces) for i in varnames]
vals = pd.DataFrame(np.array(vals).T, columns=varnames)

vals.to_csv(param_outpath, index=False)

## Plotting

Let look at the pdf of the last 50% of iterations for each parameter

In [ ]:
variables2Plot = ['fuel_x0'       , 'fuel_k',
                  'moisture_x0'   , 'moisture_k',
                  'igntions_x0'   , 'igntions_k',
                  'suppression_x0', 'suppression_k',
                  'fuel_pw'       , 'fuel_pg',
                  'cM'            , 'cMT',
                  'cP',
                  'cD1'           , 'cD2']

nvar = len(variables2Plot)
npcol = 4
nprow = np.ceil(nvar / npcol)

plt.figure(figsize=(20,5 * nprow))
def plotVar(var1, pn):
    plt.subplot(npcol, nprow, pn)
    param = vals[var1]
    
    hist, bins = np.histogram(param, bins=50)
    hist = 100.0 * hist / np.sum(hist)
    bins = bins[1:] - np.diff(bins)/2
    plt.plot(bins, hist)
    plt.xlabel(var1)
    
pn = 0
for i in variables2Plot:
    pn = pn + 1
    plotVar(i, pn)

And what to the sigmoids look like?

In [ ]:
from pdb import set_trace as browser

def pltVsFire(x, xlab, pnt = 'o', *args, **kw):
    plt.plot(x, fd.fire, pnt, alpha = 0.03, *args, **kw)
    plt.xlabel(xlab)
    
def np_sigmoid(x, k, x0):
    """
    Sigmoid function to describe limitation using tensor
    """
    return 1.0/(1.0 + np.exp(-k*(x - x0)))

def returnSigmoid(x, k, x0):
    return np_sigmoid(x, k, x0)
    
def meanParam(x, x0, k, kmult = 1.0):
    x0 = np.mean(vals[x0])
    k  = np.mean(vals[k]) * kmult

    return returnSigmoid(x, k, x0)

def randomParam(x, x0, k, kmult = 1.0, size = 100):
    ps = np.random.choice(vals.shape[0], size = size, replace = False)
    return [returnSigmoid(x, vals[k][i] * kmult, vals[x0][i]) for i in ps]

In [ ]:
plt.figure(figsize=(10, 10))

####################
## Fuel           ##
####################
plt.subplot(2, 2, 1)
## scatter plot
fp = np.mean(vals['fuel_pw'])
fg = np.mean(vals['fuel_pg'])

f_fuel = fuel_load(fd["vegCover"].values, fd["alphaMax"].values, fp, fg)
print(f_fuel.max())
pltVsFire(f_fuel, "NPP (g/m$^2$)", 'go')

## Line of best fit
Fuel = np.arange(0, f_fuel.max(), 0.01)
r_fuel = randomParam(Fuel, 'fuel_x0', 'fuel_k')
for r in r_fuel: plt.plot(Fuel, r, 'k', alpha=.01)

####################
## Moisture       ##
####################
plt.subplot(2, 2, 2)
## scatter plot
cM = np.mean(vals['cM'])
cMT = np.mean(vals['cMT'])
f_moisture = moisture(fd["alpha"].values, fd["emc"].values, fd["treecover"].values, cM, cMT)
pltVsFire(f_moisture , "Moisture = $\\alpha$ + M $\cdot$ EMC",'bo')

## Line of best fit
mst = np.arange(0.0, f_moisture.max(), 0.05)
r_moisture = randomParam(mst, 'moisture_x0', 'moisture_k', -1)
for r in r_moisture: plt.plot(mst, r, 'k', alpha=.01)


####################
## Igntions       ##
####################
plt.subplot(2, 2, 3)
## scatter plot 
cP  = np.mean(vals['cP' ])
cD1 = np.mean(vals['cD1'])
igniteMax = 10

f_ignite = ignition(fd["lightning_ignitions"].values, \
                    fd["pasture"].values, \
                    fd["population_density"].values, \
                    cP, cD1)

pltVsFire(f_ignite, "Ignitions events = Lightn + P $\cdot$ Pop Dens + D1 $\cdot$ Pasture")

## Line of best fit
Ignite = np.arange(0.0, igniteMax, 0.1)
r_Ignite = randomParam(Ignite, 'igntions_x0', 'igntions_k')
for r in r_Ignite: plt.plot(Ignite, r, 'k', alpha=.01)

plt.xlim(0, igniteMax)

####################
## Suppression    ##
####################
plt.subplot(2, 2, 4)
#scatter plot
cD2 = np.mean(vals['cD2'])
f_suppression = supression(fd["cropland"].values, \
                           fd["population_density"].values, \
                           cD2)

pltVsFire(f_suppression, "Suppression = Cropland + D2 $\cdot$ Pop den")

# Line of best fit
Suppress = np.arange(0, 100, 0.01)
r_suppression = randomParam(Suppress, 'suppression_x0', 'suppression_k', -1.0)
for r in r_suppression: plt.plot(Suppress, r, 'k', alpha=.01)

plt.xlim(0, 100)

## Sample the posterior for all data points
We now need to run the model over all datapoints and all posterior solution. We do this by boostrapping the posterior n_posterior_sample times. 

In [ ]:
n_posterior = vals.shape[1]
fd = pd.read_csv(DATAPATH)
fd['vegCover'] = (100.0 - fd['bareground'])/100.0
fd['alphaMax'] = fd['alpha_'] - 1.0

As the MCMC already introduces the required randomness for bootstrapping, we can just sample it over a simple sequance. Below, we claculate controls, limitations imposed by controls, some point soon control sensitivity, and overall burnt area for each i in the sample sequance. This is outputted into a csv file. Maybe some point I'll get it to output into a netcdf as well.

In [ ]:
ngap = int(n_posterior/n_posterior_sample)
for i in range(0, n_posterior, ngap):
    print(i)
    f_fuel     = fuel_load(fd["vegCover"].values, fd["alphaMax"].values, vals['fuel_pw'][i], vals['fuel_pg'][i])
    
    f_moisture = moisture(fd["alpha"].values, fd["emc"].values, fd["treecover"].values, vals['cM'][i], vals['cMT'][i])
    f_ignition = ignition(fd["lightning_ignitions"].values, \
                    fd["pasture"].values, \
                    fd["population_density"].values, \
                    vals['cP'][i], vals['cD1'][i])
    f_suppression = supression(fd["cropland"].values, \
                               fd["population_density"].values, \
                               vals['cD2'][i])
    
    l_fuel        = np_sigmoid(f_fuel, vals['fuel_k'][i], vals['fuel_x0'][i])
    l_moisture    = np_sigmoid(f_moisture, -vals['moisture_k'][i], vals['moisture_x0'][i])
    l_ignitions   = np_sigmoid(f_ignition, vals['igntions_k'][i], vals['igntions_x0'][i])
    l_suppression = np_sigmoid(f_suppression, -vals['suppression_k'][i], vals['suppression_x0'][i])
    prediction = vals['max_f'][i] * np.product([l_fuel, l_moisture, l_ignitions, l_suppression])
    
    PS_outFile = '../outputs/sampled_posterior_ConFire_solutions/sample_no_' + str(i) +'.csv'
    
    out = pd.DataFrame({'fuel_continuity': f_fuel,
                        'moisture_content': f_moisture,
                        'ignitions': f_ignition,
                        'suppression_index': f_suppression,
                        'fuel_limitation': l_fuel,
                        'moisture_limitation': l_moisture,
                        'ignitions_limitation': l_ignitions,
                        'suppression_limitation': l_suppression,
                        'burnt_area': prediction})
    out.to_csv(PS_outFile, index = False)